In [1]:
from prob_model import *

### Compute Payout from Odds

In [2]:
#import dataset and add one column 'payout_win'='final_tote_odds'+1
dfX_hist = pd.read_csv('df_factors_PILOT.csv')#pd.read_csv('I:/YaoTony/saleem tasks/df_factors_PILOT.csv')
dfX_hist['payout_win']=dfX_hist['final_tote_odds']+1

### Compute Probability from HDWPSRRating

In [3]:
def Score(df):
    return df

In [4]:
#if you want to analyze other factors, you just need to change 'ScoreLabel'
ScoreLabel = 'HDWPSRRating'
A = ScoreToProbViaIntegral(Score, ScoreLabel)
dfX_hist['prob_'+ScoreLabel] = dfX_hist.groupby('race_id')[ScoreLabel].transform(lambda x:A(x))

D:\Softwares\Anaconda3\envs\py35\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
D:\Softwares\Anaconda3\envs\py35\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
D:\Softwares\Anaconda3\envs\py35\lib\site-packages\scipy\stats\_distn_infrastructure.py:1738: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= self.b) & cond0
D:\Softwares\Anaconda3\envs\py35\lib\site-packages\scipy\stats\_distn_infrastructure.py:876: RuntimeWarning: invalid value encountered in greater_equal
  return (self.a <= x) & (x <= self.b)
D:\Softwares\Anaconda3\envs\py35\lib\site-packages\scipy\stats\_distn_infrastructure.py:876: RuntimeWarning: invalid value encountered in less_equal
  return (self.a <= x) & (x <= self.b)


In [5]:
#delete rows with na in prob or x8is_win
dfX_hist_noNA = dfX_hist[np.isfinite(dfX_hist['prob_'+ScoreLabel])]
dfX_hist_noNA = dfX_hist_noNA[np.isfinite(dfX_hist_noNA['x8is_win'])]

In [6]:
#normalize probability after deletinng rows
def normprob(prob):
    probnew = prob/sum(prob)
    return probnew
dfX_hist_noNA['prob_'+ScoreLabel] = dfX_hist_noNA.groupby('race_id')['prob_'+ScoreLabel].transform(lambda x:normprob(x))

### Compute Payout

In [7]:
#calculate return and advantage with different methods
factor = 'prob_'+ScoreLabel
dfX_hist_noNA['rank_' + factor] = dfX_hist_noNA.groupby("race_id")[factor].rank(ascending=False, method="min")
df = dfX_hist_noNA
attr_bench_final = factor
list_method = []
method = ['strat_double','strat_pass', 'strat_unchanged', 'strat_inverse_scaled']
bet_on_method = [factor, 'final_tote_odds']
for method1 in method:
    for method2 in method:
        for bet_on_method1 in bet_on_method:
            df_1 = compute_payout(df, attr_model=attr_bench_final, bet_amount_equal = method1, bet_amount_inequal = method2, bet_on = bet_on_method1)
            return_1 = compute_Return(df_1)
            advantage_1 = compute_advantage(df_1)
            methodall = [method1,method2,bet_on_method1,return_1,advantage_1]
            list_method.append(methodall)
df_analysis = pd.DataFrame(data = list_method, columns=['bet_amount_equal', 'bet_amount_inequal', 'bet_on','return','advantage'])


In [8]:
df_analysis

,bet_amount_equal,bet_amount_inequal,bet_on,return,advantage
0,strat_double,strat_double,prob_HDWPSRRating,-0.094220,-0.096062
1,strat_double,strat_double,final_tote_odds,-0.160643,-0.193064
2,strat_double,strat_pass,prob_HDWPSRRating,-0.093728,-0.458057
3,strat_double,strat_pass,final_tote_odds,-0.096684,-0.452359
4,strat_double,strat_unchanged,prob_HDWPSRRating,-0.094032,-0.096062
5,strat_double,strat_unchanged,final_tote_odds,-0.135797,-0.193064
6,strat_double,strat_inverse_scaled,prob_HDWPSRRating,-0.113467,-0.096062
7,strat_double,strat_inverse_scaled,final_tote_odds,-0.397214,-0.193064
8,strat_pass,strat_double,prob_HDWPSRRating,-0.095014,-0.230075
9,strat_pass,strat_double,final_tote_odds,-0.272036,-0.539669


In [10]:
dftest = compute_payout(df, attr_model=factor, bet_amount_equal = 'strat_double', bet_amount_inequal = 'strat_double', bet_on = factor)

In [12]:
dftest.loc[:,["rank_finish_tote_odds","rank_prob_HDWPSRRating","bet_amount"]]

D:\Softwares\Anaconda3\envs\py35\lib\site-packages\pandas\core\indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


,rank_finish_tote_odds,rank_prob_HDWPSRRating,bet_amount
0,NaN,3.0,0.0
1,NaN,5.0,0.0
2,NaN,6.0,0.0
3,NaN,2.0,0.0
4,NaN,1.0,2.0
5,NaN,7.0,0.0
6,NaN,4.0,0.0
7,NaN,5.0,0.0
8,NaN,6.0,0.0
9,NaN,2.0,0.0
